In [1]:
import sys
sys.path.append("../")
import pandas as pd
import datetime as dt
from api.oanda_api import OandaApi
from dateutil import parser
import timeit
import plotly.graph_objects as go
from technicals import trend_checker
from technicals import zone_detector
from technicals import pattern_detector
pd.set_option('display.max_columns', None)

In [2]:
df_an = pd.read_pickle('../data/GBP_USD_H4.pkl').copy()
def apply_technicals(df):
    df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
    trend_checker.apply_downtrend(df)
    df['bullish_strength_score'] = df.apply(pattern_detector.bullish_strength, axis=1)
    #need to make this not return a df but add in place
    df = pattern_detector.detect_bottom_reversal_setups(df)
    zone_detector.attach_zones_to_confirmations(df)

apply_technicals(df_an)
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,sTime,ma_10,ma_50,in_downtrend,bullish_strength_score
49,2015-01-14 02:00:00+00:00,1130,1.51678,1.51705,1.51456,1.51637,1.51669,1.51696,1.51443,1.51626,1.51688,1.51714,1.51466,1.51648,s15-01-14 02:00,1.515886,1.519332,True,0.195
50,2015-01-14 06:00:00+00:00,5879,1.51631,1.52242,1.51464,1.51736,1.51620,1.52220,1.51456,1.51724,1.51642,1.52264,1.51473,1.51747,s15-01-14 06:00,1.515977,1.518572,True,0.178
51,2015-01-14 10:00:00+00:00,6306,1.51735,1.52696,1.51626,1.52570,1.51724,1.52684,1.51616,1.52560,1.51746,1.52711,1.51636,1.52580,s15-01-14 10:00,1.516829,1.517991,True,0.801
52,2015-01-14 14:00:00+00:00,5540,1.52569,1.52687,1.52016,1.52214,1.52561,1.52677,1.52006,1.52205,1.52577,1.52697,1.52026,1.52223,s15-01-14 14:00,1.517263,1.517474,True,0.218
53,2015-01-14 18:00:00+00:00,2008,1.52212,1.52375,1.52106,1.52344,1.52204,1.52367,1.52098,1.52320,1.52221,1.52383,1.52114,1.52368,s15-01-14 18:00,1.518017,1.517109,False,0.570


In [ ]:
df_an['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df_an.time]
trend_checker.apply_downtrend(df_an)
df_an.tail(20)
support_levels, resistance_levels = zone_detector.find_support_resistance(df_an)
zones = zone_detector.get_zones_for_price(1.01934, support_levels, resistance_levels)
print(zones)

In [ ]:
df_bull = df_an.copy()
df_bull['bullish_strength_score'] = df_bull.apply(pattern_detector.bullish_strength, axis=1)
strength_threshold = 0.7
df_bull['strong_bullish'] = df_bull['bullish_strength_score'] >= strength_threshold
df_bull.head()


In [ ]:
df_signals = pattern_detector.detect_bottom_reversal_setups(df_bull)
zone_detector.attach_zones_to_confirmations(df_signals)
df_signals = df_signals[df_signals['setup_stage'] == 'confirmation']
df_signals.shape

In [ ]:
fig = trend_checker.highlight_downtrend_candles(df_an)
x_start = df_an['sTime'].iloc[0]
x_end = df_an['sTime'].iloc[-1]
y_min = df_an['mid_l'].min()
y_max = df_an['mid_h'].max()
padding = (y_max - y_min) * 0.1  # 10% vertical padding

stage_colors = {
    'bottom': 'blue',
    'breakout': 'orange',
    'reentry': 'purple',
    'confirmation': 'green'
}

# Add a trace for each setup stage
for _, row in df_signals.iterrows():
    stage = row['setup_stage']
    fig.add_trace(go.Scatter(
        x=[row['sTime']],
        y=[row['mid_c']],  # or mid_l if you want the low of the candle
        mode='markers+text',
        marker=dict(color=stage_colors.get(stage, 'black'), size=10, symbol='x'),
        name=stage,
        text=[stage],
        textposition='top center',
        showlegend=False  # Set to True if you want to display multiple legends
    ))

# # Plot support levels
# for level in support_levels:
#     fig.add_trace(go.Scatter(
#         x=[x_start, x_end],
#         y=[level, level],
#         mode='lines',
#         name=f'Support {level}',
#         line=dict(color='green', width=1)
#     ))

#     # Plot resistance levels
# for level in resistance_levels:
#     fig.add_trace(go.Scatter(
#         x=[x_start, x_end],
#         y=[level, level],
#         mode='lines',
#         name=f'Resistance {level}',
#         line=dict(color='red', width=1)
#     ))

fig.update_layout(
    yaxis=dict(
        range=[y_min - padding, y_max + padding],
        fixedrange=False  # Allow zooming
    )
)

fig.show()